## Harmony Py Library
### Job Results STAC data

In [ ]:
import sys
sys.path.append('..')
from harmony import BBox, Client, Collection, Request


First, let's get a job processing in Harmony.

In [ ]:
harmony_client = Client()  # assumes .netrc usage

collection = Collection(id='C1234088182-EEDTEST')
request = Request(
    collection=collection,
    spatial=BBox(-165, 52, -140, 77)
)

job_id = harmony_client.submit(request)
job_id


Harmony-py can return the STAC Catalog URL for a completed job.

In [ ]:
stac_catalog_url = harmony_client.stac_catalog_url(job_id, show_progress=True)


Following the directions for PySTAC (https://pystac.readthedocs.io/en/latest/quickstart.html), we can hook our harmony-py client into STAC_IO.

In [ ]:
from urllib.parse import urlparse
import requests
from pystac import STAC_IO

def requests_read_method(uri):
    parsed = urlparse(uri)
    if parsed.hostname.startswith('harmony.'):
        return harmony_client.read_text(uri)
    else:
        return STAC_IO.default_read_text_method(uri)

STAC_IO.read_text_method = requests_read_method

In [ ]:
from pystac import Catalog

cat = Catalog.from_file(stac_catalog_url)